In [1]:
#Import libraries:
import pandas as pd
import random
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
train = pd.read_csv("D:/DADOS USUARIO/Documents/springboard/capstone project/Instacart Kaggle/train.csv", header=0)

In [3]:
train = train.reset_index()

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20592325 entries, 0 to 20592324
Data columns (total 24 columns):
index                              int64
order_number                       int64
prod_user_times                    int64
n_times_prod_ordered               int64
n_times_prod_reordered             int64
count_order_id                     int64
min_order_number                   int64
max_order_number                   int64
mean_add_to_cart_order             float64
times_prod_ordered                 int64
prod_reorder_probability           float64
prod_reorder_times                 float64
prod_reorder_ratio                 float64
user_mean_days_since_prior         float64
user_orders                        int64
user_period                        float64
user_total_prod                    int64
user_reorder_ratio                 float64
distinct                           int64
user_average_basket                float64
up_order_rate                      float64
up_ord

In [5]:
#due to memory error i´ll reduce the train data in 50%
train = train.sample(frac=0.01)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205923 entries, 18267516 to 8352522
Data columns (total 24 columns):
index                              205923 non-null int64
order_number                       205923 non-null int64
prod_user_times                    205923 non-null int64
n_times_prod_ordered               205923 non-null int64
n_times_prod_reordered             205923 non-null int64
count_order_id                     205923 non-null int64
min_order_number                   205923 non-null int64
max_order_number                   205923 non-null int64
mean_add_to_cart_order             205923 non-null float64
times_prod_ordered                 205923 non-null int64
prod_reorder_probability           205923 non-null float64
prod_reorder_times                 205923 non-null float64
prod_reorder_ratio                 205923 non-null float64
user_mean_days_since_prior         205923 non-null float64
user_orders                        205923 non-null int64
user_period     

In [6]:
target = 'reordered'
IDcol = 'index'

In [7]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult)
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['reordered'], eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(dtrain['reordered'].values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['reordered'], dtrain_predprob))
    print(cvresult)##add this because the funtion was not returning the best estimator
    
    feat_imp = pd.Series(alg._Booster.get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')


In [8]:
#Choose all predictors except target & IDcols
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
 learning_rate =0.3,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, train, predictors)

TypeError: 'DataFrame' object cannot be interpreted as an integer

In [ ]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.3, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_


In [30]:
cvresult

NameError: name 'cvresult' is not defined

In [49]:
pip install coveralls --use-mirrors


The following command must be run outside of the IPython shell:

    $ pip install coveralls --use-mirrors

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more informations on how to install packages:

    https://docs.python.org/3/installing/


In [3]:
columns_train = list(range(23))

In [5]:
train.columns = columns_train

In [9]:
# split data into X and y
X_train = train.iloc[:,0:22]
y_train = train.iloc[:,22]

In [12]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,1,1,2452,1615,280530,1,1,21.0,2452,1.087216,...,18.000000,3,36.0,76,0.141732,58,25.333333,93510.000000,2,93510.000000
1,7,1,35633,27749,2505309,7,7,23.0,35633,1.162354,...,22.083333,13,265.0,210,0.044099,142,16.153846,192716.076923,6,357901.285714
2,7,1,2272,1220,1771198,7,7,7.0,2272,0.792776,...,4.437500,81,355.0,768,0.015543,220,9.481481,21866.641975,74,23615.973333
3,5,1,8010,4530,795223,5,8,13.0,8010,0.769540,...,21.000000,12,231.0,195,0.087921,88,16.250000,66268.583333,4,99402.875000
4,7,2,8010,4530,795223,5,8,13.0,8010,0.769540,...,21.000000,12,231.0,195,0.087921,88,16.250000,66268.583333,4,99402.875000


In [13]:
#parameters for the model
param = {'max_depth': 6, \
         'eta': 0.1, \
         'objective': "reg:logistic",\
         "eval_metric" : "logloss", \
         "min_child_weight" : 10, \
         "gamma" : 0.70, \
         "subsample" : 0.76, \
         "colsample_bytree" : 0.95, \
         "alpha" : 2e-05, \
         "lambda" : 10}
num_round = 80

In [14]:
dtrain = xgb.DMatrix(X_train, label=y_train)
algo = xgb.train(param, dtrain, num_round)

In [18]:
# make predictions for test data
dtest = xgb.DMatrix(X_test)
preds = algo.predict(dtest)

In [19]:
test['reordered'] = (preds > 0.21) * 1

In [20]:
cols = ['product_id','order_number','prod_user_times','n_times_prod_ordered','n_times_prod_reordered','count_order_id','min_order_number','max_order_number','mean_add_to_cart_order','times_prod_ordered','prod_reorder_probability','prod_reorder_times','prod_reorder_ratio','user_mean_days_since_prior','user_orders','user_period','user_total_prod','user_reorder_ratio','distinct','user_average_basket','order_id','up_order_rate','up_orders_since_last_order','up_order_rate_since_first_order','reordered']
test.columns = cols

In [21]:
test.head()

,product_id,order_number,prod_user_times,n_times_prod_ordered,n_times_prod_reordered,count_order_id,min_order_number,max_order_number,mean_add_to_cart_order,times_prod_ordered,...,user_period,user_total_prod,user_reorder_ratio,distinct,user_average_basket,order_id,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,reordered
0,48171,3,1,6925,3552,1491849,3,3,12.0,6925,...,109.0,70,0.110599,46,11.666667,1416320,248641.500000,3,372962.25,0
1,5209,1,1,17,6,1321974,1,6,10.5,17,...,76.0,204,0.075855,133,22.666667,1735923,146886.000000,3,146886.00,0
2,5209,6,2,17,6,1321974,1,6,10.5,17,...,76.0,204,0.075855,133,22.666667,1735923,146886.000000,3,146886.00,0
3,42625,5,1,13378,5865,1830981,5,6,16.5,13378,...,171.0,184,0.110199,90,20.444444,707453,203442.333333,3,366196.20,0
4,42625,6,2,13378,5865,1830981,5,6,16.5,13378,...,171.0,184,0.110199,90,20.444444,707453,203442.333333,3,366196.20,0


# Submission file

In [16]:
#remove order_id and product_id
X_test = test.drop([test.columns[0], test.columns[20]], axis=1)
columns_test = list(range(22))
X_test.columns = columns_test
X_test.head()

In [3]:
test = pd.read_csv("D:/DADOS USUARIO/Documents/springboard/capstone project/Instacart Kaggle/test.csv", header=0)

In [22]:
t = test.loc[:,['order_id','product_id', 'reordered']]

In [23]:
t = t[t.reordered == 1]
t = t.drop('reordered', axis=1)
t['product_id'] = t.product_id.astype(str)

In [24]:
t = t.groupby('order_id').agg(lambda x: ' '.join(set(x)))

In [25]:
t = t.reset_index()
t = t.rename(index=str, columns={"product_id": "products"})

In [27]:
test_orders = pd.read_csv("D:/DADOS USUARIO/Documents/springboard/capstone project/Instacart Kaggle/orders.csv")
test_orders = test_orders[test_orders.eval_set == 'test']

In [28]:
order_id = pd.Series(test.order_id.unique())
order_id = test_orders[~test_orders.order_id.isin(t.order_id)]
miss = pd.DataFrame({'order_id': order_id.order_id,
                    'products': "None"})

In [29]:
submission = t.append(miss)

In [30]:
submission = submission.set_index('order_id')

In [31]:
submission.sort_index()

,products
order_id,
17,39275 38777 21463 13107
34,39180 44663 47766 2596 43504 21137 16083 39475...
137,29594 23794 24852 41787 5134 44422 2326 25890 ...
182,47672 5479 39322 11520 13629 26128 31737 8021 ...
257,27966 12456 24852 29837 13870 49235 21137 2203...
313,30391 46906 13198 47626 12779 45007 28535 3496...
353,21137 35561 33000 40688
386,25513 45066 21479 37935 4920 2326 30450 40759 ...
414,27705 14947 48988 36865 33320 21709 31730 4463...


In [32]:
submission.to_csv('submission.csv')